In [ ]:
import networkx as nx
from roaringbitmap import RoaringBitmap
import matplotlib.pyplot as plt
import time
import csv
import heapq as hq
from itertools import product

Algoritmo de branch and bound com paralelismo de bits

In [ ]:
def smallest_last_degree(G):

	residual_degree = {}
	Q = []
	for v in G.nodes:
		residual_degree[v] = len( G.adj[v] )
		Q.append( (residual_degree[v], v) )

	hq.heapify(Q)

	S = []
	while len(Q) > 0:
		(degree, v) = hq.heappop(Q)
		if v in S:
			continue
		#print(degree)
		S.append(v)
		for u in G.adj[v]:
			if u not in S:
				residual_degree[u] -= 1
				hq.heappush(Q, (residual_degree[u], u))
	S.reverse()
	return S

In [ ]:
def max_kpartite(adj, V, l):
    index = 1
    R = V.copy()
    sizeR = len(R)

    while sizeR > 0 and index <= l:
        S = R.copy()
        while len(S) > 0:
            v = S.min()
            S -= adj[v]
            R.remove(v)
            S.remove(v)
            sizeR -= 1
        index += 1
    return R

In [ ]:
def bitcliquePartite(G, V, clique_atual, best_clique, number_of_nodes):
    number_of_nodes[0] += 1
    sizeV = len(V)

    if sizeV == 0:
        if len(clique_atual) > len(best_clique):
            best_clique[:] = clique_atual[:]
    else:
        l = len(best_clique) - len(clique_atual)
        R = max_kpartite(G, V, l)

        order = list(R)

        m = len(order)
        for i in range(m-1, -1, -1):
            v = order[i]
            V2 = V.copy()
            V2 &= G[v]
            V.remove(v)
            clique_atual.append(v)
            bitcliquePartite(G, V2, clique_atual, best_clique, number_of_nodes)
            clique_atual.pop()

In [ ]:
def algoBitCliqueMaxPartite(G):
	print("branch and bound roar")
	start_time = time.time()

	S = smallest_last_degree(G)
	rotulo = {}
	for i in range( len(S) ):
		rotulo[ S[i] ] = i
	G = nx.relabel_nodes(G, rotulo)

	adj = []
	n = len(G.nodes)
	for i in range( n ):
		adj.append(  RoaringBitmap( list( G.adj[i])  ) )

	clique_atual = []
	best_clique = []
	V = RoaringBitmap( [ i for i in range(n) ])
	number_of_nodes = [ 0 ]


	bitcliquePartite( adj ,  V , clique_atual, best_clique, number_of_nodes )

	end_time = time.time()
	elapsed_time = end_time - start_time
	print(f"Elapsed time: {elapsed_time:.6f} seconds")
	print("best_clique: ",  best_clique)
	print("size: ", len(best_clique) )
	print("number of nodes: ", number_of_nodes[0])
	return best_clique, number_of_nodes, elapsed_time

Algoritmo de branch and bound com DLTS

Comparação entre os Algoritmos de branch and bound com paralelismo de bits e com DLTS

In [ ]:
data = [ "nodes", "densidade", "DLTS Clique (tempo)" , "algoBitCliqueMaxPartite (tempo)", "DLTS Clique (nodes)", "algoBitCliqueMaxPartite (nodes)"]
with open('exato.csv', 'w', encoding='UTF8', newline='') as f:
  writer = csv.writer(f)
  writer.writerow(data)

valores1 = [10, 15, 20, 30]
valores2 = [0.25, 0.5, 0.75, 0.9]

combinacoes = list(product(valores1, valores2))
for c in combinacoes:
  for i in range(10):
    nodes = c[0]
    densidade = c[1]
    G = nx.gnp_random_graph(nodes, densidade)
    best_clique1, number_of_nodes1, elapsed_time1 = DLTSClique(G)
    best_clique2, number_of_nodes2, elapsed_time2 = algoBitCliqueMaxPartite(G)

    data = [ nodes, densidade, elapsed_time1, elapsed_time2, number_of_nodes1[0], number_of_nodes2[0] ]
    with open('exato.csv', 'a', encoding='UTF8', newline='') as f:
      writer = csv.writer(f)
      writer.writerow(data)

In [ ]:
with open('exato.csv', 'r', encoding='UTF8') as f:
    reader = csv.reader(f)
    header = next(reader)

    dados_agrupados = []
    current_group = []
    i=0
    for row in reader:
        # Converte os valores numéricos de string para float
        converted_row = []
        for j, valor in enumerate(row):
            if j == 2 or j == 3 or j == 4:
                converted_row.append(float(valor))
            elif j == 5 or j == 6:
                converted_row.append(int(valor))
        current_group.append(converted_row)
        i+=1
        
        # A cada 10 linhas salva na lista
        if i % 10 == 0:
            dados_agrupados.append(current_group)
            current_group = []

medias_agrupadas = []
for grupo in dados_agrupados:
    media_grupo = []
    for col in range(5):
        coluna_valores = [linha[col] for linha in grupo]
        media_grupo.append(round(sum(coluna_valores) / len(coluna_valores), 2))
    medias_agrupadas.append(media_grupo)

with open('medias.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    header = ["nodes", "densidade", "DLTS Clique (tempo medio)", "algoBitCliqueMaxPartite (tempo medio)","DLTS Clique (nodes medio)", "algoBitCliqueMaxPartite (nodes medio)",]
    writer.writerow(header)
    for i, (nodes, densidade) in enumerate(combinacoes):
        data = [
            nodes,
            densidade,
            medias_agrupadas[i][0],
            medias_agrupadas[i][1],
            medias_agrupadas[i][2],
            "{:.2e}".format(medias_agrupadas[i][3]),
            "{:.2e}".format(medias_agrupadas[i][4])
        ]
        writer.writerow(data)

In [ ]:
x = []
y = []
y2 = []
for i in range(40):
    if i<10:
        x.append(0.25)
    elif i<20:
        x.append(0.5)
    elif i<30:
        x.append(0.75)
    elif i<40:
        x.append(0.9)

with open('exato.csv', newline='') as csvfile:
    # Cria um leitor CSV
    csvreader = csv.reader(csvfile, delimiter=',')
    # Itera sobre as linhas do arquivo
    next(csvreader)
    for row in csvreader:
        # Adiciona a linha como um dicionário à lista de dados
        y.append(round(float(row[2]), 2))
        y2.append(round(float(row[3]), 2))

plt.scatter(x, y, color='orange', label='BB')
plt.scatter(x, y2, color='blue', label='BR')
plt.xlabel('Densidade')
plt.ylabel('tempo (seg)')
plt.title('Comparativo entre DLTS e BitClique')
plt.legend()
plt.yticks([0, 25, 50, 75, 125])
plt.xticks([0, 0.25, 0.5, 0.75, 0.9])
plt.savefig('grafico.png')